In [1]:
!pip install open3d

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 23.7 MB/s eta 0:00:00
  Using cached Pillow-10.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 19.6 MB/s eta 0:00:00m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 32.9 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 2.2.2
    Uninstalling Werkzeug-2.2.2:
      Successfully uninstalled Werkzeug-2.2.2
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.7.3
    Uninstalling nbformat-5.7.3:
      Successfully uninstalled nbformat-5.7.3


In [1]:
import open3d as o3d
import numpy as np
from scipy.io import savemat

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Load the stl file
mesh = o3d.io.read_triangle_mesh('normalized_mesh9.stl')
mesh.compute_vertex_normals()

# fit to unit cube
mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()),
           center=mesh.get_center())
o3d.visualization.draw_geometries([mesh])


In [3]:
# Voxelize the stl file
print('voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh,
                                                              voxel_size=0.009)
print(voxel_grid)
o3d.visualization.draw_geometries([voxel_grid])

voxelization
VoxelGrid with 46408 voxels.


In [39]:
# Create the 3D matrix from voxels
voxel_data = voxel_grid.get_voxels()
voxel_matrix = np.zeros((128, 128, 128), dtype=int)
for idx in voxel_data:
    x, y, z = idx.grid_index
    voxel_matrix[x, y, z] = 1

savemat('voxel_matrix.mat', {'volume': voxel_matrix})

/tmp/ipykernel_13036/871901202.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  voxel_matrix = np.zeros((128, 128, 128), dtype=np.int)


In [44]:
import meshlib.mrmeshnumpy as mrn
import meshlib.mrmeshpy as mr 

vol = voxel_matrix.astype(np.double)
slice_to_plot = vol - vol.min() / vol.max() - vol.min()
simple_vol = mrn.simpleVolumeFrom3Darray(slice_to_plot)
float_grid = mr.simpleVolumeToDenseGrid(simple_vol)

mesh = mr.gridToMesh(float_grid, mr.Vector3f(0.1, 0.1, 0.1), 0.5)

mr.saveMesh(mesh, f'normalized_mesh_tooth.stl')


In [45]:
mesh = o3d.io.read_triangle_mesh('normalized_mesh_tooth.stl')
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

# Convert NII to MAT file

In [1]:
!pip install nibabel as nib

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [4]:
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.io as sio
import os

# Loading the files
source_folder = "voxel_data/Cropped"
destination_folder = "voxel_data/processed_data/" 
# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# List all files in the source folder
nii_files = [file for file in os.listdir(source_folder) if file.endswith('.nii')]

for nii_file in nii_files:
    nii_file_path = os.path.join(source_folder, nii_file)
    nii_img = nib.load(nii_file_path)
    nii_data = nii_img.get_fdata()
    
    # Create a dictionary to hold the data
    data_dict = {'tensor': nii_data}
    
    # Define the destination MATLAB file path
    mat_file_path = os.path.join(destination_folder, os.path.splitext(nii_file)[0] + '.mat')
    
    # Save the data to MATLAB file
    sio.savemat(mat_file_path, data_dict)
    
    print(f"Converted {nii_file} to {os.path.basename(mat_file_path)} with shape : {nii_data.shape}")

print("Conversion completed.")

Converted NidhiSeg60.nii to NidhiSeg60.mat with shape : (128, 128, 128)
Converted NidhiSeg41.nii to NidhiSeg41.mat with shape : (128, 128, 128)
Converted RekhaSegB146.nii to RekhaSegB146.mat with shape : (128, 128, 128)
Converted AruSegB163.nii to AruSegB163.mat with shape : (128, 128, 128)
Converted AruSegB114.nii to AruSegB114.mat with shape : (128, 128, 128)
Converted NidhiSeg48.nii to NidhiSeg48.mat with shape : (128, 128, 128)
Converted RekhaSegB133.nii to RekhaSegB133.mat with shape : (128, 128, 128)
Converted AruSegB18.nii to AruSegB18.mat with shape : (128, 128, 128)
Converted NidhiSeg75.nii to NidhiSeg75.mat with shape : (128, 128, 128)
Converted NidhiSeg55.nii to NidhiSeg55.mat with shape : (128, 128, 128)
Converted RekhaSegB148.nii to RekhaSegB148.mat with shape : (128, 128, 128)
Converted NidhiSeg134.nii to NidhiSeg134.mat with shape : (128, 128, 128)
Converted NidhiSeg103.nii to NidhiSeg103.mat with shape : (128, 128, 128)
Converted RekhaSegB123.nii to RekhaSegB123.mat wit

KeyboardInterrupt: 